In [1]:
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

import time
import os

from bs4 import BeautifulSoup
import re
import pandas as pd

import requests
import os
import urllib
from urllib.parse import urlparse

import subprocess

In [2]:
# root_url = 'https://classroom.udacity.com/nanodegrees/nd787/dashboard/overview'
root_url = 'https://classroom.udacity.com/nanodegrees/nd892/dashboard/overview'
# root_folder = './fcnd_downloaded'
root_folder = './nlpnd'
system_download_folder_path = "/Users/huohsien/Downloads"

IMPLICIT_DELAY = 2
EXPLICIT_DELAY = 6

In [3]:
#setup option for chrome profile
chrome_options = Options()
chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
chrome_options.add_experimental_option("prefs", prefs)

#start web driver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.implicitly_wait(IMPLICIT_DELAY)

wait = WebDriverWait(driver, EXPLICIT_DELAY)

[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [4]:
# start
driver.get(root_url)

# wait for the main page to finish loading
dummy = wait.until(ec.visibility_of_element_located((By.XPATH, "//ol[@id='tree-core-curriculum']")))
time.sleep(0.5)

In [5]:
topic = []
topic_name = ""
lesson_name = ""
lesson_part_name = ""
page_folder_path = ""

In [6]:
def elm_xpath(base, xpath_str):
    elm = base.find_element_by_xpath(xpath_str)
    return elm

def elms_xpath(base, xpath_str):
    elms = base.find_elements_by_xpath(xpath_str)
    return elms

def get_html(elm):
    html = elm.get_attribute("innerHTML")
    return html

def get_href(elm):
    href = elm.get_attribute("href")
    return href

In [7]:
def is_gpu_workspace():
    wait = WebDriverWait(driver, 3)
    try:
        dummy = wait.until(ec.visibility_of_element_located((By.XPATH, "//section[@class='modal-gpu-fetch--toggle--title--2yZN0']")))
    except TimeoutException:
        wait = WebDriverWait(driver, EXPLICIT_DELAY)
        return False
    wait = WebDriverWait(driver, EXPLICIT_DELAY)
    return True


In [8]:
def is_workspace():
    wait = WebDriverWait(driver, 6)
    try:
#         for terminal workspace such as in Flying Car ND
#         dummy = wait.until(ec.visibility_of_element_located((By.XPATH, "//div[@class='panel files-panel']")))
        dummy = wait.until(ec.visibility_of_element_located((By.XPATH, "//div[@class='index--jupyter-container--2Dbj9']")))
    except TimeoutException:
        wait = WebDriverWait(driver, EXPLICIT_DELAY)
        return False
    print("workspace!")
    wait = WebDriverWait(driver, EXPLICIT_DELAY)
    return True

In [9]:
def create_folders():
    
    dirName = os.path.join(root_folder, topic_name, lesson_name, lesson_part_name)
    dirName = re.sub(':', '：', dirName)
    
    try:
        # Create target Directory
        os.makedirs(dirName)
    except FileExistsError:
        None
    return dirName

In [10]:
def download_css_files():
    
    css_folder_path = os.path.join(root_folder, topic_name,lesson_name, lesson_part_name, "css")
    css_folder_path = re.sub(':', '：', css_folder_path)
    
    if not os.path.exists(css_folder_path):
        os.makedirs(css_folder_path)
        
    html_src = driver.page_source
    base = driver.current_url
    soup = BeautifulSoup(html_src)
    css_link_elms = soup.find_all('link', {'rel': 'stylesheet'})
    
    css_links = []
    for idx, css_link_elm in enumerate(css_link_elms, start = 1):
        link = css_link_elm['href']
        if link.startswith('/'):
            link = urllib.parse.urljoin(base, link)
        css_links.append(link)
#         print(idx, ": ", link)
    
    for css_link in css_links:
        r = requests.get(css_link, allow_redirects=True)
        a = urlparse(css_link)
        file_name = os.path.basename(a.path)
        file_name = os.path.join(css_folder_path, file_name)
#         print("file_name= ", file_name) 
    
        open(file_name, 'wb').write(r.content)

In [11]:
def make_images_local(soup):
    
    images_folder = os.path.join(root_folder, topic_name,lesson_name, lesson_part_name, "img")
    images_folder = re.sub(':', '：', images_folder)
    
    img_elms = soup.find_all('img')
    for img_elm in img_elms:
        img_link = img_elm['src']
        r = requests.get(img_link, allow_redirects=True)
        a = urlparse(img_link)
        file_name = os.path.basename(a.path)                
        img_path = "./" + images_folder + "/" + file_name

        if not os.path.exists(images_folder):
            os.makedirs(images_folder)

        open(img_path, 'wb').write(r.content)
#         print("file_name: ", file_name)
        # replace src in img element
        img_elm['src'] = "./img/{}".format(file_name)

def create_head_str():
    
    css_folder_path = os.path.join(root_folder, topic_name,lesson_name, lesson_part_name, "css")
    css_folder_path = re.sub(':', '：', css_folder_path)
    
    css_filenames = [f for f in os.listdir(css_folder_path) if os.path.isfile(os.path.join(css_folder_path, f))]
    
    page_html = driver.page_source
    page_soup = BeautifulSoup(page_html, 'html.parser')
    style_elms = page_soup.find('head').find_all('style')
    
    meta_elm = '<meta http-equiv="Content-type" content="text/html; charset=utf-8">'
    head_str = '<head>' + meta_elm
    for css_filename in css_filenames:
        link_str = '<link rel="stylesheet" href="./css/{}">'.format(css_filename)
        head_str = head_str + link_str
    for style_elm in style_elms:
        head_str = head_str + str(style_elm)
    
    head_str = head_str + '</head>'
#     print("head_str: ", head_str)
    return head_str

def process_content_html(content_html_str):
    
    download_css_files()
            
    head_str = create_head_str()
    body_str = embed_tag('body', content_html_str)
    page_html_str = head_str + body_str
    html_str = embed_tag('html', page_html_str)
    
    # remove unwanted div element
    soup = BeautifulSoup(html_str, 'html.parser')
    div_to_be_remove_class_name = '_main--notification-bar--2AVbT'
    to_be_removed_elms = soup.find_all('div',class_=div_to_be_remove_class_name)
    for to_be_removed_elm in to_be_removed_elms:
        to_be_removed_elm.decompose()
    
    make_images_local(soup)
    
    return soup.prettify()

def copy_youtube_link_to_clipboard(video_link):
    write_to_clipboard(video_link)
    time.sleep(1)
    
def grab_all_youtube_links(content_html_str, download=False):

    content_soup = BeautifulSoup(content_html_str, 'html.parser')
    youtube_iframe_elms = content_soup.find_all('iframe', {'title': 'YouTube video player'})
    video_links = ''
    for idx, youtube_iframe_elm in enumerate(youtube_iframe_elms, start=1):
        video_link = youtube_iframe_elm['src']
        video_link = video_link + '#index={}_{}_{}_{}'.format(topic_name,lesson_name, lesson_part_name,idx)
        video_links = video_links + video_link + '\n\n'
        if download == True:
            copy_youtube_link_to_clipboard
        
    with open('all_youtube_video_links.txt','a') as fp:
        fp.writelines(video_links)

In [12]:
def save_processed_html():

    content_html = driver.page_source
    html_str = content_html
    
    download_css_files()
    
    # remove unwanted div element
    soup = BeautifulSoup(html_str, 'html.parser')
    div_to_be_remove_class_name = '_main--notification-bar--2AVbT'
    to_be_removed_elms = soup.find_all('div',class_=div_to_be_remove_class_name)
    for to_be_removed_elm in to_be_removed_elms:
        to_be_removed_elm.decompose()
    
    make_images_local(soup)
    
    grab_all_youtube_links(content_html, False)
    
    file_name = os.path.join(root_folder, topic_name,lesson_name, lesson_part_name, "index.html")
    file_name = re.sub(':', '：', file_name)
    print("file_name= ", file_name)
    with open(file_name, 'w') as f:
        f.write(html_str)

# Loop the 3rd level

In [13]:
def loop_lesson_part():
    print("**** Enter: loop_lesson_part() ****")
    
    global lesson_part_name
    global page_folder_path
    
    lis =  elms_xpath(driver, "//ol[@class='index--contents-list--33vHB']/li")

    for i in range(len(lis)):
#     for i in [1, 2, 3]:
#     for i in [17]:
        
        li = lis[i]

        a = elm_xpath(li, ".//a")
        lesson_part_name = a.text
      
        a.click()
        print("\t\t\t", lesson_part_name)

#         # check if it is Udacity cloud workspace. when the workspace is shutdowned, it will ask if you want to enable gpu mode
#         if is_gpu_workspace():
#             btn = driver.find_element_by_xpath("//div[@class='modal-gpu-fetch--toggle--options--3sCYv']/button[2]")
#             btn.click()
            
        # check if it is Udacity cloud workspace. if so click open sidemenu
        if is_workspace():
            time.sleep(5)
            btn = driver.find_element_by_xpath("//button[@class='vds-round-button vds-round-button--minimal-inverse vds-round-button--icon']")
            btn.click()
            time.sleep(1) # for tester to view which is important in development and debugging stage. can be removed in production stage
    
        # write a file to file structure
#         page_folder_path = create_folders()
#         key = input("stop?")
#         if key == "y":
#             NOT_DEFINED_FUNC()
        
        save_processed_html()
        
            
        lis =  elms_xpath(driver, "//ol[@class='index--contents-list--33vHB']/li")

    # leave lesson part level and go back to lesson level
    
#     for i in range(len(lis)):
#         driver.back()

#     for i in [14,15, 16]:
#         driver.back()

    time.sleep(0.5)
    driver.find_element_by_xpath("//h3[@class='_sidebar--header--2BaH_']/a").click()
    time.sleep(0.5)
    print("**** Leave: loop_lesson_part() ****")

# Loop the 2nd level

In [14]:
def loop_lessons():
    print("**** Enter: loop_lessons() ****")
    
    global lesson_name
    
    lesson_elms =  elms_xpath(driver, "//div[@class='_syllabus--container--30zm9 shared--outer-container--3eppq']/ol/li")

    for i in range(len(lesson_elms)):
#     for i in [10]:
        lesson_elm = lesson_elms[i]
        
        a = elm_xpath(lesson_elm, ".//a")
        
        # deal with collapse ui for reasons such as completed
        if a.get_attribute("class") == "_lesson-collapsed--curtain--1KsEA shared--curtain--2_FSP":
            # this lession is completed and ui is collapsed. need one more click
            a.click()
            time.sleep(0.1)
            a = elm_xpath(lesson_elm, ".//a")
        
        h2 = elm_xpath(lesson_elm, "./div/div/div/div/div[1]/div[1]/div[1]/h2")
        lesson_name = "Lesson {}. {}".format(i+1, h2.text)
        
        a.click()
        print("\t\t", lesson_name)
        
#         time.sleep(1) #simulation doing something here
        loop_lesson_part()
        
        lesson_elms =  elms_xpath(driver, "//div[@class='_syllabus--container--30zm9 shared--outer-container--3eppq']/ol/li")
    
    # leave lesson lesson level and go back to topic level
    

    print("**** Leave: loop_lessons() ****")

# Loop the 1st level

In [15]:
def loop_topics():
    print("**** Enter:loop_topics() ****")
    
    global topic_name
    
    topic_elms =  elms_xpath(driver, "//ol[@id='tree-core-curriculum']/li")

    for i in range(len(topic_elms)):
#     for i in [0]:
        topic_elm = topic_elms[i]
        
        a = elm_xpath(topic_elm, ".//a")
        span = elm_xpath(a, "./div/span[2]")
        topic_name = "Topic: {}. {}".format(i+1, span.text)

        a.click()
        print("\t", topic_name)

        # loop through lessons in this topic
        loop_lessons()

        topic_elms =  elms_xpath(driver, "//ol[@id='tree-core-curriculum']/li")
        
    print("**** Leave:loop_topics() ****")

In [16]:
loop_topics()

**** Enter:loop_topics() ****
	 Topic: 1. Introduction to Natural Language Processing
**** Enter: loop_lessons() ****
		 Lesson 1. Welcome to Natural Language Processing
**** Enter: loop_lesson_part() ****
			 1. Welcome to the Natural Language Processing Nanodegree
file_name=  ./nlpnd/Topic： 1. Introduction to Natural Language Processing/Lesson 1. Welcome to Natural Language Processing/1. Welcome to the Natural Language Processing Nanodegree/index.html
			 2. Community Guidelines
file_name=  ./nlpnd/Topic： 1. Introduction to Natural Language Processing/Lesson 1. Welcome to Natural Language Processing/2. Community Guidelines/index.html
**** Leave: loop_lesson_part() ****
		 Lesson 2. Mentor Help, Peer Chat, and Careers
**** Enter: loop_lesson_part() ****
			 1. What It Takes
file_name=  ./nlpnd/Topic： 1. Introduction to Natural Language Processing/Lesson 2. Mentor Help, Peer Chat, and Careers/1. What It Takes/index.html
			 2. Getting Help
file_name=  ./nlpnd/Topic： 1. Introduction to 

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=90.0.4430.212)
